In [1]:
import pandas as pd
import os
import datetime
from dotenv import dotenv_values
import snowflake.connector as sc
import snowflake.connector.pandas_tools as snp
import re

In [12]:
config = dotenv_values(os.path.join(os.getcwd(),'env','gpc','.env'))

con = sc.connect(
    user = config["USERNAME"],
    password = config["PASSWORD"],
    account = config["ACCOUNT"],
    warehouse = config["WAREHOUSE"],
    role = config["ROLE"],
    database = config["TARGET_DB"],
    schema = config["TARGET_SCHEMA"]
)
# Create a cursor object
cur = con.cursor()

In [13]:
# Fetch the column names
query = f"""
SELECT column_name from CDM_DATALAKE.information_schema.columns
    where 
        table_name = 'GPC_DEID_TUMOR' 
        and table_schema = 'GPC'
        and lower(column_name) like lower('%_N%') 
        and lower(column_name) not like lower('RAW_%')
        and data_type = 'TEXT';
"""
cur.execute(query)
columns = cur.fetchall()
columns_to_exclude = {'DATE_CASE_INITIATED_N2085','DATE_CASE_COMPLETED_N2090', 'PATID'}
column_list = [col[0] for col in columns if col[0] not in columns_to_exclude]

# # Construct the list of column names
column_list_str = ", ".join(column_list)
column_list_str

In [14]:
# Construct the unpivot SQL query

# Table name
table_name = 'TUMOR_FACT'
# source = 'DEIDENTIFIED_PCORNET_CDM.CDM.DEID_TUMOR'
source = 'CDM_DATALAKE.GPC.GPC_DEID_TUMOR'

unpivot_query = f"""
CREATE OR REPLACE VIEW {table_name} AS
(
    select 
        -1 :: NUMBER(38, 0)                                                                              as ENCOUNTER_NUM
        ,pc.PATIENT_NUM                                                                                  as PATIENT_NUM
        ,concat('NAACCR|', split_part(concept, '_N', -1), ':', coalesce(concept_cd, ''))                 as CONCEPT_CD
        ,'@'                                                                                             as PROVIDER_ID
        ,DATE_CASE_INITIATED_N2085 :: TIMESTAMP                                                          as START_DATE
        ,'@'                                                                                             as MODIFIER_CD
        ,1                                                                                               as INSTANCE_NUM
        ,''                                                                                              as VALTYPE_CD
        ,''                                                                                              as TVAL_CHAR
        ,cast(null as  integer)                                                                          as NVAL_NUM
        ,''                                                                                              as VALUEFLAG_CD
        ,cast(null as  integer)                                                                          as QUANTITY_NUM
        ,'@'                                                                                             as UNITS_CD
        ,cast(DATE_CASE_COMPLETED_N2090 as TIMESTAMP)                                                    as END_DATE
        ,'@'                                                                                             as LOCATION_CD
        ,cast(null as  text)                                                                             as OBSERVATION_BLOB
        ,cast(null as  integer)                                                                          as CONFIDENCE_NUM
        ,CURRENT_TIMESTAMP                                                                               as UPDATE_DATE
        ,CURRENT_TIMESTAMP                                                                               as DOWNLOAD_DATE
        ,CURRENT_TIMESTAMP                                                                               as IMPORT_DATE
        ,cast(null as VARCHAR(50))                                                                       as SOURCESYSTEM_CD
        ,cast(null as  integer)                                                                          as UPLOAD_ID
    from {source}
    unpivot (
        concept_cd for concept IN ({column_list_str})
    )
  
    inner join {config["TARGET_SCHEMA"]}.patient_crosswalk as pc
    using (patid)
)
union all
(
    select 
        -1 :: NUMBER(38, 0)                                                                              as ENCOUNTER_NUM
        ,pc.PATIENT_NUM                                                                                  as PATIENT_NUM
        ,concat('NAACCR|MORPH:',histologic_type_icd_o3_n522, '/',behavior_code_icd_o3_n523)              as CONCEPT_CD
        ,'@'                                                                                             as PROVIDER_ID
        ,DATE_CASE_INITIATED_N2085 :: TIMESTAMP                                                          as START_DATE
        ,'@'                                                                                             as MODIFIER_CD
        ,1                                                                                               as INSTANCE_NUM
        ,''                                                                                              as VALTYPE_CD
        ,''                                                                                              as TVAL_CHAR
        ,cast(null as  integer)                                                                          as NVAL_NUM
        ,''                                                                                              as VALUEFLAG_CD
        ,cast(null as  integer)                                                                          as QUANTITY_NUM
        ,'@'                                                                                             as UNITS_CD
        ,cast(DATE_CASE_COMPLETED_N2090 as TIMESTAMP)                                                    as END_DATE
        ,'@'                                                                                             as LOCATION_CD
        ,cast(null as  text)                                                                             as OBSERVATION_BLOB
        ,cast(null as  integer)                                                                          as CONFIDENCE_NUM
        ,CURRENT_TIMESTAMP                                                                               as UPDATE_DATE
        ,CURRENT_TIMESTAMP                                                                               as DOWNLOAD_DATE
        ,CURRENT_TIMESTAMP                                                                               as IMPORT_DATE
        ,cast(null as VARCHAR(50))                                                                       as SOURCESYSTEM_CD
        ,cast(null as  integer)                                                                          as UPLOAD_ID
    from {source}
    inner join {config["TARGET_SCHEMA"]}.patient_crosswalk as pc
    using (patid)
    
)   
;
"""
unpivot_query

In [15]:
# Execute the unpivot SQL query
# cur.execute(f"DROP TABLE IF EXISTS {database_name}.{schema_name}.{table_name}")
cur.execute(unpivot_query)

# Close the cursor and connection
cur.close()
con.close()

print("View created successfully.")

In [16]:
run_con = sc.connect(
    user = config["USERNAME"],
    password = config["PASSWORD"],
    account = config["ACCOUNT"],
    warehouse = config["WAREHOUSE"],
    role = config["ROLE"],
    database = config["TARGET_DB"],
    schema = config["METADATA_SCHEMA"]
)
# Create a cursor object
run_cur = run_con.cursor()

run_cur.execute(f"call RUNTOTALNUM( 'TUMOR_FACT', 'I2B2DATA', 'NAACCR_ONTOLOGY')")